In [4]:
import os
import pandas as pd
import numpy as np

In [5]:
# Tian 1:10, 17:26
subcortical_index = list(range(0,10)) + list(range(16,26))
# Schaefer: 
# lh-mPFC: 199:205
# rh-mPFC: 464:470
# lh-Ins: 67, 108:111, 126:128
# rh-Ins: 319, 361:364, 383:386
## ACC: 390
# Glasser
cortical_roi = ['lh_dlPFC', 'rh_dlPFC', 'lh_mPFC', 'rh_mPFC', 'lh_PCC', 'rh_PCC', 'lh_Ins', 'rh_Ins']
lh_dlPFC_index = [205, 246, 247, 249, 250, 252, 262, 263, 264, 265, 266, 276, 277]
rh_dlPFC_index = [25, 66, 67, 69, 70, 72, 82, 83, 84, 85, 86, 96, 97]
lh_mPFC_index = [236, 237, 238, 239, 240, 241, 242, 243, 244, 248, 267, 343, 344, 345, 358, 359]
rh_mPFC_index = [56, 57, 58, 59, 60, 61, 62, 63, 64, 68, 87, 163, 164, 165, 178, 179]
lh_PCC_index = [193, 194, 206, 209, 210, 211, 212, 213, 214, 300, 321, 340, 341]
rh_PCC_index = [13, 14, 26, 29, 30, 31, 32, 33, 34, 120, 141, 160, 161]
lh_Ins_index = [285, 287, 288, 289, 290, 291, 293, 294, 346, 347, 348, 357]
rh_Ins_index = [105, 107, 108, 109, 110, 111, 113, 114, 166, 167, 168, 177]
dic_cortical_roi = {
    'lh_dlPFC': lh_dlPFC_index,
    'rh_dlPFC': rh_dlPFC_index,
    'lh_mPFC': lh_mPFC_index,
    'rh_mPFC': rh_mPFC_index,
    'lh_PCC': lh_PCC_index,
    'rh_PCC': rh_PCC_index,
    'lh_Ins': lh_Ins_index,
    'rh_Ins': rh_Ins_index
}

In [6]:
parcellation_dir = '/Users/xiaoqianxiao/tool/parcellation'
sub_cortical_filename = 'Tian_Subcortex_S2_3T_label.csv'
sub_cortical_file_path = os.path.join(parcellation_dir, sub_cortical_filename)
df_sub_cortical_name = pd.read_csv(sub_cortical_file_path, header=None)
df_sub_cortical_roi_name_need = df_sub_cortical_name.iloc[subcortical_index,0]
df_cortical_roi_name = pd.DataFrame(cortical_roi)
df_roi_name = pd.concat([df_cortical_roi_name, df_sub_cortical_roi_name_need],ignore_index=True)
df_roi_name.columns = ['roi_name']
from itertools import combinations
combinations_list = list(combinations(df_roi_name['roi_name'], 2))
df_roi_combinations = pd.DataFrame(combinations_list, columns=['ROI1', 'ROI2'])
print(df_roi_combinations)

             ROI1          ROI2
0        lh_dlPFC      rh_dlPFC
1        lh_dlPFC       lh_mPFC
2        lh_dlPFC       rh_mPFC
3        lh_dlPFC        lh_PCC
4        lh_dlPFC        rh_PCC
..            ...           ...
373     lh_THA-VA  lh_NAc-shell
374     lh_THA-VA   lh_NAc-cole
375     lh_THA-DA  lh_NAc-shell
376     lh_THA-DA   lh_NAc-cole
377  lh_NAc-shell   lh_NAc-cole

[378 rows x 2 columns]


In [7]:
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure


# === Step 1: Define Functions === #

def load_dataset(base_dir, data_set):
    """
    Load dataset-specific files.
    """
    fMRIinfo_file_path = os.path.join(base_dir, f"{data_set}_data_set.csv")
    participant_file_path = os.path.join(base_dir, "participants_fMRI.csv")
    return pd.read_csv(fMRIinfo_file_path), pd.read_csv(participant_file_path)


def load_subject_timeseries(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index):
    """
    Load cortical and subcortical timeseries data for a subject.
    """
    cortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Glasser.csv.gz"
    cortical_file_path = os.path.join(derivatives_dir, "timeseries", cortical_file_name)

    subcortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Tian_Subcortex_S2_3T.csv.gz"
    subcortical_file_path = os.path.join(derivatives_dir, "timeseries", subcortical_file_name)

    if not (os.path.exists(cortical_file_path) and os.path.exists(subcortical_file_path)):
        print(f"Missing files for subject {subject_ID}, session {session_ID}.")
        return None

    # Load and process cortical timeseries
    df_cortical_all = pd.read_csv(cortical_file_path, compression="gzip", index_col=0, header=0)
    df_cortical_roi = pd.DataFrame({
        roi: df_cortical_all.iloc[dic_cortical_roi[roi]].mean(axis=0)
        for roi in dic_cortical_roi.keys()
    })

    # Load and process subcortical timeseries
    df_subcortical_all = pd.read_csv(subcortical_file_path, compression="gzip", index_col=0, header=0)
    df_subcortical_roi = df_subcortical_all.iloc[subcortical_index]

    # Combine cortical and subcortical ROIs
    return pd.concat([df_cortical_roi, df_subcortical_roi.transpose()], axis=1)


def clean_data(data):
    """
    Handle missing values and remove constant features from time series data.
    """
    # Fill NaNs with column-wise means
    data_filled = np.copy(data)
    for j in range(data.shape[1]):
        if np.isnan(data[:, j]).any():
            data_filled[:, j] = np.nan_to_num(data[:, j], nan=np.nanmean(data[:, j]))

    # Remove constant features
    non_constant_features = data_filled[:, data_filled.std(axis=0) != 0]
    return non_constant_features


def compute_connectivity(data):
    """
    Compute connectivity matrix for the given time series data.
    """
    correlation_measure = ConnectivityMeasure(kind="correlation")
    return correlation_measure.fit_transform([data])[0]


def extract_upper_triangle(matrix):
    """
    Extract the upper triangle values (excluding diagonal) from a connectivity matrix.
    """
    upper_tri_indices = np.triu_indices(matrix.shape[0], k=1)
    return matrix[upper_tri_indices]


# === Step 2: Define the Pipeline === #

def process_fMRI_subject(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index):
    """
    Full pipeline for processing a single subject's fMRI data.
    """
    # Load subject timeseries
    df_roi = load_subject_timeseries(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index)
    if df_roi is None:
        return None, None

    # Clean data
    #cleaned_data = clean_data(df_roi.values)

    # Standardize data
    standardized_data = StandardScaler().fit_transform(df_roi.values)

    # Compute connectivity matrix
    connectivity_matrix = compute_connectivity(standardized_data)

    # Extract upper triangle
    upper_triangle = extract_upper_triangle(connectivity_matrix)

    return upper_triangle, subject_ID


def process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index):
    """
    Full pipeline for processing fMRI data for all subjects.
    """
    # Set paths
    base_dir = os.path.join(user_dir, project_name, "data")
    derivatives_dir = os.path.join(base_dir, "derivatives")

    # Load dataset
    df_fMRIinfo, df_participants = load_dataset(base_dir, data_set)
    subject_IDs = df_fMRIinfo["eid"].unique()

    # Initialize lists for data
    connectivity_data = []
    subject_ids_cleaned = []

    # Process each subject individually
    for subject_ID in subject_IDs:
        upper_triangle, cleaned_subject_ID = process_fMRI_subject(
            subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index
        )
        if upper_triangle is not None:
            connectivity_data.append(upper_triangle)
            subject_ids_cleaned.append(cleaned_subject_ID)

    # Filter participants based on available data
    df_filtered = df_participants.loc[df_participants["eid"].isin(subject_ids_cleaned)]

    return np.array(connectivity_data), df_filtered


In [8]:
#codes for modeling
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
# Perform stratified 10-Fold Cross-Validation
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Ensure binary target
def ensure_binary_target(y):
    unique_values = np.unique(y)
    if len(unique_values) > 2:
        raise ValueError("Target variable contains more than two classes. Please preprocess the data.")
    if unique_values.dtype == bool:
        return y.astype(int)
    elif set(unique_values) == {0, 1} or set(unique_values) == {1, 0}:
        return y
    else:
        raise ValueError("Target variable is not binary. Please preprocess the data.")


from sklearn.model_selection import train_test_split
# Split data into training and testing sets while preserving class distribution
def split_data(X, y, N_random_state, test_size=0.2):
    """
    Split the data into training and testing sets while preserving class ratios.
    
    Parameters:
    - X: Features.
    - y: Target labels.
    - test_size: Proportion of the dataset to include in the test split.
    - random_state: Random state for reproducibility.
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    return train_test_split(X, y, test_size=test_size, random_state=N_random_state, stratify=y)



# Model selection using cross-validation
def model_selection(X_train, y_train):
    models = {
        "Logistic Regression": LogisticRegression(),  # Provides coefficients (coef_)
        #"Ridge Classifier": LogisticRegression(penalty='l2', solver='liblinear'),  # coef_
        #"Lasso (L1)": LogisticRegression(penalty='l1', solver='liblinear'),  # coef_
        #"LDA": LinearDiscriminantAnalysis(),  # Provides coefficients (coef_)
        #"Perceptron": Perceptron(),  # Provides coefficients (coef_)
        #"SVM (Linear)": SVC(kernel='linear'),  # Provides coefficients (coef_) when kernel='linear'
    }

    best_model = None
    best_score = -np.inf
    best_name = ""

    for model_name, model in models.items():
        cv_score = cross_val_score(model, X_train, y_train, cv=stratified_kfold, scoring='accuracy').mean()
        print(f"Model: {model_name}, CV Score: {cv_score:.4f}")

        if cv_score > best_score:
            best_score = cv_score
            best_model = model
            best_name = model_name

    print(f"Best Model: {best_name} with CV score: {best_score:.4f}")
    return best_model, best_name


def feature_selection_with_ChiSquare(X, target, n_features):
    from sklearn.feature_selection import SelectKBest, chi2
    # Apply Chi-Square Test
    X_shifted = X - X.min() + 1e-9
    selector = SelectKBest(chi2, k=n_features)  # Select top 2 features
    X_selected = selector.fit_transform(X_shifted, target)
    
    # Get boolean mask of selected features
    selected_features_mask = selector.get_support()
    
    print("Selected Features Mask:", selected_features_mask)
    print("Scores:\n", selector.scores_)
    return selected_features_mask

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

def feature_selection_with_rfe(X_train, y_train, n_features, best_model):
    """
    Perform feature selection using RFE, with fallback to univariate selection for models without coefficients.
    """
    if hasattr(best_model, "coef_") or hasattr(best_model, "feature_importances_"):
        # Use RFE for models with coefficients or feature importances
        rfe = RFE(estimator=best_model, n_features_to_select=n_features, step=1)
        rfe.fit(X_train, y_train)

        selected_features = rfe.support_
        if np.sum(selected_features) == 0:
            print("No features selected using RFE. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    else:
        # Fallback to univariate feature selection
        print("Model lacks coefficients/feature importance; using univariate feature selection.")
        
        # Use SelectKBest with F-statistic (or mutual information if preferred)
        selector = SelectKBest(score_func=f_classif, k=n_features)
        selector.fit(X_train, y_train)

        selected_features = selector.get_support()
        if np.sum(selected_features) == 0:
            print("No features selected using univariate method. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    return selected_features

from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import numpy as np

def feature_selection_with_rfe_cv(X_train, y_train, best_model, scoring_metric='accuracy'):
    """
    Perform feature selection using RFE or univariate selection, optimizing the number of features automatically
    using cross-validation.
    """
    
    def evaluate_features(model, X, y, num_features):
        """
        Helper function to evaluate the model's performance with the given number of features using cross-validation.
        """
        if hasattr(model, "coef_") or hasattr(model, "feature_importances_"):
            # Perform RFE with the given number of features
            rfe = RFE(estimator=model, n_features_to_select=num_features, step=1)
            X_selected = rfe.fit_transform(X, y)
        else:
            # Use univariate feature selection as a fallback
            selector = SelectKBest(score_func=f_classif, k=num_features)
            X_selected = selector.fit_transform(X, y)

        # Evaluate model performance using cross-validation
        scores = cross_val_score(model, X_selected, y, cv=stratified_kfold, scoring=scoring_metric)
        return scores.mean()

    # Iterate over a range of features to find the optimal number of features
    best_score = -np.inf
    optimal_num_features = 0
    for num_features in range(1, X_train.shape[1] + 1):
        score = evaluate_features(best_model, X_train, y_train, num_features)
        if score > best_score:
            best_score = score
            optimal_num_features = num_features

    print(f"Optimal number of features: {optimal_num_features} with cross-validated score: {best_score:.4f}")

    # Perform final RFE or univariate selection with the optimal number of features
    if hasattr(best_model, "coef_") or hasattr(best_model, "feature_importances_"):
        rfe = RFE(estimator=best_model, n_features_to_select=optimal_num_features, step=1)
        rfe.fit(X_train, y_train)
        selected_features = rfe.support_
    else:
        selector = SelectKBest(score_func=f_classif, k=optimal_num_features)
        selector.fit(X_train, y_train)
        selected_features = selector.get_support()

    return selected_features, optimal_num_features


# Two-step grid search for hyperparameter optimization
def tune_model_hyperparameters(model, model_name, X_train, y_train):
    refined_grid = {}  # Initialize with a default value to avoid "unbound variable" error

    if model_name == "Logistic Regression":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Ridge Classifier":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Lasso (L1)" or model_name == "ElasticNet (L1+L2)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "SVM (Linear)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Random Forest":
        broad_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
    elif model_name == "Perceptron":
        broad_param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}
    elif model_name == "LDA":
        broad_param_grid = {'shrinkage': [None, 'auto'], 'solver': ['svd', 'lsqr', 'eigen']}
    else:
        raise ValueError(f"Model {model_name} does not have a defined parameter grid.")

    # Broad Grid Search
    broad_search = GridSearchCV(model, broad_param_grid, cv=stratified_kfold, scoring='accuracy', verbose=1, n_jobs=-1)
    broad_search.fit(X_train, y_train)
    best_params_broad = broad_search.best_params_

    # Define refined grid based on broad search results
    if model_name in ["Logistic Regression", "Lasso (L1)", "ElasticNet (L1+L2)", "SVM (Linear)"]:
        refined_grid = {'C': np.linspace(best_params_broad['C'] * 0.1, best_params_broad['C'] * 10, 5)}
    elif model_name == "Random Forest":
        refined_grid = {
            'n_estimators': [max(10, best_params_broad['n_estimators'] - 50), best_params_broad['n_estimators'], best_params_broad['n_estimators'] + 50],
            'max_depth': [None] if not best_params_broad['max_depth'] else [
                max(1, best_params_broad['max_depth'] - 5), best_params_broad['max_depth'], best_params_broad['max_depth'] + 5]
        }
    elif model_name == "Perceptron":
        refined_grid = {'alpha': np.linspace(best_params_broad['alpha'] * 0.1, best_params_broad['alpha'] * 10, 5)}
    elif model_name == "LDA":
        refined_grid = {'shrinkage': [best_params_broad['shrinkage']], 'solver': [best_params_broad['solver']]}

    # Narrow Grid Search
    narrow_search = GridSearchCV(model, refined_grid, cv=stratified_kfold, scoring='accuracy', verbose=1, n_jobs=-1)
    narrow_search.fit(X_train, y_train)

    print(f"Best Parameters (Broad Search): {best_params_broad}")
    print(f"Best Parameters (Narrow Search): {narrow_search.best_params_}")

    best_model = narrow_search.best_estimator_
    return best_model


from sklearn.metrics import roc_auc_score
# Train and evaluate final model
def train_and_evaluate_final_model(X_train, y_train, X_test, y_test, model):
    """
    Train and evaluate the final model. Reports accuracy and AUC.
    Parameters:
    - X_train: Features for training.
    - y_train: Labels for training.
    - X_test: Features for testing.
    - y_test: Labels for testing.
    - model: Machine learning model (must support `fit` and `predict_proba`).

    Returns:
    - model: Trained model.
    - test_accuracy: Accuracy on the test set.
    - test_auc: AUC score on the test set.
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate accuracy
    test_accuracy = model.score(X_test, y_test)
    
    # Predict probabilities for AUC computation
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
        test_auc = roc_auc_score(y_test, y_proba)
        print(f"Test Set AUC (Final Model): {test_auc:.4f}")
    else:
        print("Model does not support probability predictions; skipping AUC computation.")
        test_auc = None

    print(f"Test Set Accuracy (Final Model): {test_accuracy:.4f}")
    y_pred = model.predict(X_test)
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    return model, test_accuracy, test_auc



# Cosine similarity between two models
from sklearn.metrics.pairwise import cosine_similarity

# Extract the weights (coefficients) or feature importances
def get_model_vector(model):
    if hasattr(model, 'coef_'):  # Linear models with coefficients
        return model.coef_.flatten()
    elif hasattr(model, 'feature_importances_'):  # Tree-based models
        return model.feature_importances_
    else:
        raise ValueError(f"Model of type {type(model)} does not have coefficients or feature importances.")
    
def calculate_model_similarity(vector1, vector2):
    """
    Calculate the similarity between two models using cosine similarity or feature importances.

    Parameters:
    - model1: First trained model
    - model2: Second trained model

    Returns:
    - similarity: Cosine similarity score between the two models' coefficients or importances.
    """
    try:
        # Ensure vectors are of the same length
        if len(vector1) != len(vector2):
            raise ValueError("Model vectors have different lengths. Ensure the models were trained on the same features.")

        # Calculate cosine similarity
        similarity = cosine_similarity([vector1], [vector2])
        return similarity[0][0]  # Return the scalar similarity value

    except ValueError as e:
        print(f"Error in calculating similarity: {e}")
        return None
    
def evaluate_model_on_datasets(model, selected_features, datasets, dataset_names, model_name):
    """
    Evaluates the given model on multiple datasets.
    
    Parameters:
    - model: The trained model to be evaluated.
    - selected_features: The indices of selected features used by the model.
    - datasets: List of (X, y) pairs for evaluation.
    - dataset_names: List of dataset names corresponding to datasets.
    - model_name: Name of the model (used for logging).
    """
    print(f"\nEvaluating {model_name} on other datasets:")
    for (X, y), dataset_name in zip(datasets, dataset_names):
        X_selected = X[:, selected_features]
        y_pred = model.predict(X_selected)
        accuracy = accuracy_score(y, y_pred)
        print(f"\nAccuracy of {model_name} on {dataset_name}: {accuracy:.4f}")
        
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_selected)[:, 1]
            auc = roc_auc_score(y, y_proba)
            print(f"{dataset_name} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
        else:
            print(f"{dataset_name} - Accuracy: {accuracy:.4f}")
            auc = None
        
        print("Confusion Matrix:\n", confusion_matrix(y, y_pred))
        print("Classification Report:\n", classification_report(y, y_pred))

def delong_roc_test(y_true, auc1, auc2):
    se = ((auc1 * (1 - auc1)) + (auc2 * (1 - auc2))) / len(y_true)
    z = (auc1 - auc2) / (se ** 0.5)
    p_value = norm.sf(abs(z)) * 2  # Two-tailed test
    return p_value

# Full pipeline
def pipeline1(X, y, n_features, N_random_state):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y, N_random_state)

    best_model, best_name = model_selection(X_train, y_train)

    selected_features = feature_selection_with_rfe(X_train, y_train, n_features, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    tuned_model = tune_model_hyperparameters(best_model, best_name, X_train_selected, y_train)

    final_model, test_accuracy, test_auc = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )
    return final_model, selected_features, test_accuracy

def pipeline2(X, y, N_random_state):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y, N_random_state)

    best_model, best_name = model_selection(X_train, y_train)

    
    selected_features, features_number = feature_selection_with_rfe_cv(X_train, y_train, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    tuned_model = tune_model_hyperparameters(best_model, best_name, X_train_selected, y_train)

    final_model, test_accuracy, test_auc = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )
    return final_model, selected_features, test_accuracy

def compare_models_and_analyze_topography1(X_data_set1, y_data_set1, X_data_set2, y_data_set2, X_data_set3, y_data_set3, X_data_set4, y_data_set4, n_features, N_random_state):
    print("Training data_set1 model...")
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline1(X_data_set1, y_data_set1, n_features, N_random_state)
    weight_type1 = get_model_vector(final_model_data_set1)
    print("\nTraining data_set2 model...")
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline1(X_data_set2, y_data_set2, n_features, N_random_state)
    weight_type2 = get_model_vector(final_model_data_set2)
    print("\nTraining data_set3 model...")
    final_model_data_set3, selected_features_data_set3, test_accuracy_data_set3 = pipeline1(X_data_set3, y_data_set3, n_features, N_random_state)
    weight_type3 = get_model_vector(final_model_data_set3)
    print("\nTraining data_set4 model...")
    final_model_data_set4, selected_features_data_set4, test_accuracy_data_set4 = pipeline1(X_data_set4, y_data_set4, n_features, N_random_state)
    weight_type4 = get_model_vector(final_model_data_set4)
    print("\nEvaluating data_set1 model on other data_sets:")
    
    # Datasets and corresponding names
    datasets = [(X_data_set1, y_data_set1), (X_data_set2, y_data_set2), 
                (X_data_set3, y_data_set3), (X_data_set4, y_data_set4)]
    dataset_names = ["data_set1", "data_set2", "data_set3", "data_set4"]
    
    # Evaluate models trained on each dataset
    evaluate_model_on_datasets(final_model_data_set1, selected_features_data_set1, datasets[1:], dataset_names[1:], "data_set1 model")
    evaluate_model_on_datasets(final_model_data_set2, selected_features_data_set2, datasets[:1] + datasets[2:], ["data_set1"] + dataset_names[2:], "data_set2 model")
    evaluate_model_on_datasets(final_model_data_set3, selected_features_data_set3, datasets[:2] + datasets[3:], ["data_set1", "data_set2", "data_set4"], "data_set3 model")
    evaluate_model_on_datasets(final_model_data_set4, selected_features_data_set4, datasets[:3], dataset_names[:3], "data_set4 model")
    
        
    return final_model_data_set1, final_model_data_set2, final_model_data_set3, final_model_data_set4,selected_features_data_set1, selected_features_data_set2, selected_features_data_set3, selected_features_data_set4, weight_type1, weight_type2, weight_type3, weight_type4

def compare_models_and_analyze_topography2(X_data_set1, y_data_set1, X_data_set2, y_data_set2, N_random_state):
    print("Training data_set1 model...")
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline2(X_data_set1, y_data_set1, N_random_state)
    # X_data_set1_selected = X_data_set1[:, selected_features_data_set1]
    # y_pred = final_model_data_set1.predict(X_data_set1_selected)
    # print("Confusion Matrix:\n", confusion_matrix(y_data_set1, y_pred))
    # print("Classification Report:\n", classification_report(y_data_set1, y_pred))
    
    print("\nTraining data_set2 model...")
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline2(X_data_set2, y_data_set2, N_random_state)
    # X_data_set2_selected = X_data_set2[:, selected_features_data_set2]
    # y_pred2 = final_model_data_set2.predict(X_data_set2_selected)
    # print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred2))
    # print("Classification Report:\n", classification_report(y_data_set2, y_pred2))
    

    print("\nEvaluating data_set1 model on data_set2 dataset:")
    X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
    y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
    accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
    print(f"Accuracy of data_set1 model on data_set2 data: {accuracy:.4f}")
    # Check if the model supports probability predictions for AUC computation
    if hasattr(final_model_data_set1, "predict_proba"):
        y_proba_data_set2 = final_model_data_set1.predict_proba(X_data_set2_selected)[:, 1]  # Probabilities for the positive class
        auc = roc_auc_score(y_data_set2, y_proba_data_set2)
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    else:
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}")
        auc = None  # AUC not computed due to lack of probability support

    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred_data_set2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred_data_set2))

    print("\nCalculating cosine similarity between data_set1 and data_set2 model weights:")
    try:
        similarity = calculate_model_similarity(final_model_data_set1, final_model_data_set2)
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity:.4f}")
    except ValueError as e:
        similarity = "N/A (Model type not compatible for cosine similarity)"
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity}")
    return similarity, test_accuracy_data_set1

In [9]:
# Define user inputs
user_dir = "/Users/xiaoqianxiao"
project_name = "UKB"
session_ID = 2  # Specify session

In [10]:
data_set = "ah"  # Dataset identifier
X_ah, df_ah = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_ah = df_ah["active_history"]

data_set = "ih"  # Dataset identifier
X_ih, df_ih = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_ih = df_ih["inactive_history"]

data_set = "a_noh"  # Dataset identifier
X_a_noh, df_a_noh = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_a_noh = df_a_noh["active_no_history"]

Missing files for subject 1529291, session 2.


In [11]:
from itertools import compress
filtered_list_ah = list(compress(X_ah, y_ah))
my_array = np.array(filtered_list_ah)
data = my_array

In [15]:
best_model = LogisticRegression(solver='liblinear', random_state=42)
selected_features = feature_selection_with_rfe_cv(X_ah, y_ah, best_model, scoring_metric='roc_auc')

Optimal number of features: 4 with cross-validated score: 0.6139


In [23]:
def pipeline_FeatureSelection(X, y, n_features, N_random_state):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y, N_random_state)

    best_model, best_name = model_selection(X_train, y_train)

    selected_features = feature_selection_with_rfe(X_train, y_train, n_features, best_model)
    
    return selected_features

In [32]:
selected_features = pipeline_FeatureSelection(X_ah, y_ah, 30, 422)

Model: Logistic Regression, CV Score: 0.5310
Best Model: Logistic Regression with CV score: 0.5310
Model lacks coefficients/feature importance; using univariate feature selection.
